In [1]:
import re
import string
import joblib
import json

In [10]:
import torch

In [2]:
with open("./data/merged_dialogue.txt", "r", encoding="utf-8") as f:
    merged_data = f.readlines()
merged_data[:10]

['中濑[line]闭嘴，你这跳梁小丑！\n',
 '伦太郎[line]放开我，你们......唔？\n',
 '???[line]......\n',
 '伦太郎[line]啊......\n',
 '牧濑红莉栖[line]能跟我来一下吗？\n',
 '伦太郎[line]你、你是机关的人！？\n',
 '牧濑红莉栖[line]哈？\n',
 '伦太郎[line]若机关势力已渗透至此，那我的战略就存在致命漏洞......\n',
 '牧濑红莉栖[line]别胡闹了，跟我来。\n',
 '伦太郎[line]......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？\n']

In [3]:
chunks = []
dia_text = ""
for seq in merged_data:
    if seq != "<SEP>\n":
        dia_text += seq
    else:
        chunks.append(dia_text.strip())
        dia_text = ""

print(chunks[0])

中濑[line]闭嘴，你这跳梁小丑！
伦太郎[line]放开我，你们......唔？
???[line]......
伦太郎[line]啊......
牧濑红莉栖[line]能跟我来一下吗？
伦太郎[line]你、你是机关的人！？
牧濑红莉栖[line]哈？
伦太郎[line]若机关势力已渗透至此，那我的战略就存在致命漏洞......
牧濑红莉栖[line]别胡闹了，跟我来。
伦太郎[line]......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？
牧濑红莉栖[line]你到底在说什么？只是有几个问题要问你。
伦太郎[line]你以为我会乖乖就范？我早看透机关的伎俩了。
牧濑红莉栖[line]机关到底是什么东西？
伦太郎[line]是我。遭遇机关特工拦截。......对，是牧濑红莉栖。危险等级极高。......不必，我自有脱身之——
牧濑红莉栖[line]......
伦太郎[line]你做什么！？
牧濑红莉栖[line]你的手机......根本没开机啊。
伦太郎[line]......
牧濑红莉栖[line]......刚才在和谁通话？
伦太郎[line]这、这种精神攻击对我是无效的！告诉你也无妨，此非普通手机，离手即刻自毁。桀哈哈哈！
牧濑红莉栖[line]......也就是说你在自言自语。
伦太郎[line]咳！
牧濑红莉栖[line]十五分钟前你想对我说什么？
伦太郎[line]不知所云。
牧濑红莉栖[line]发布会开始前十五分钟。你当时想对我说什么吧？神情异常激动。简直像是下一秒就要哭出来似的。为什么？我们以前见过吗？还有，你为什么知道我的名字？
伦太郎[line]凤凰院凶真的智慧没有边界。天才少女啊，下次相见便是敌手了！
牧濑红莉栖[line]哈啊？
伦太郎[line]永别了！呜哈哈哈哈哈！该死的机关！居然派出她这样的特工，看来是动真格了！但我还不能被他们抓住。...糟了。我把真由理落下了。嗯？有邮件？...奇怪？可恶，真由理怎么不接电话？等等，难道说！那个蛇蝎美人绑架了真由理！？卑鄙！这就是机关的作风吗！？必须回去救她。呵，看来被我吓跑了。也罢，这次就放她一马。真由理！为什么不接电话？我们走。
真由理[line]冈伦！我的金属乌帕逃走了。
伦太郎[line]逃走？这玩意儿是活的？太扯了吧。
真由理[line]可能掉在什么地方了...
伦太郎

# 把助手的回答长度多于filter_size个字符(不包括标点)筛选出来。向上添加，直到添加到上一轮对话角色是助手

In [4]:
punctuation = string.punctuation + "，。！？、；：“”‘’（）《》【】—"
print(punctuation)
pattern = re.compile(f"[{punctuation}]")

s = "啊？你在说什么......"
re.sub(pattern, "", s)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~，。！？、；：“”‘’（）《》【】—


'啊你在说什么'

In [5]:
filter_chunks = []
filter_size = 10

for chunk in chunks:
    start_idx = 0
    splited_chunk = chunk.split("\n")
    
    # 将[line]转换为 :
    # colon_chunk = chunk.replace("[line]", ":").split("\n")

    for i, seq in enumerate(splited_chunk):
        splited = seq.split("[line]")
        character, lines = splited
        if character == "牧濑红莉栖": 
            if len(re.sub(pattern, "", lines)) > filter_size:
                filter_chunks.append(splited_chunk[start_idx: i + 1])
                start_idx = i + 1
            else:
                start_idx = i + 1
filter_chunks

[['伦太郎[line]......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？',
  '牧濑红莉栖[line]你到底在说什么？只是有几个问题要问你。'],
 ['伦太郎[line]咳！', '牧濑红莉栖[line]十五分钟前你想对我说什么？'],
 ['伦太郎[line]不知所云。',
  '牧濑红莉栖[line]发布会开始前十五分钟。你当时想对我说什么吧？神情异常激动。简直像是下一秒就要哭出来似的。为什么？我们以前见过吗？还有，你为什么知道我的名字？'],
 ['冈部伦太郎[line]听好了桶子！我有预感电话微波炉（暂定名称）正是开启命运石之门的关键！',
  '桥田至[line]啥石头门？从你说『惊世谬论』开始我就跟不上了。',
  '???[line]呀！对、对不起！',
  '冈部伦太郎[line]什…！？你...你竟然...',
  '牧濑红莉栖[line]这位先生，有什么事吗？好痛！'],
 ['冈部伦太郎[line]为...为什么...你应该死了才对！为什么会在这里！？而且身上...连道伤口都没有...',
  '牧濑红莉栖[line]快松手！很痛啊！你精神有问题吗？'],
 ['桥田至[line]又来了？',
  '冈部伦太郎[line]『又』是什么意思？',
  '桥田至[line]一周前你不是发邮件说这事吗？',
  '冈部伦太郎[line]我发的邮件？胡说！三小时前我才目睹她的死亡！',
  '牧濑红莉栖[line]可以别擅自讨论别人的生死吗？我活得好好的。'],
 ['冈部伦太郎[line]我只想确认真相。', '牧濑红莉栖[line]确认需要袭胸吗！？变态！去死吧！？'],
 ['桥田至[line]路易丝酱名台词再现！',
  '冈部伦太郎[line]我亲眼所见绝无虚假！',
  '牧濑红莉栖[line]等等！你该不会看到我的内...'],
 ['冈部伦太郎[line]吾之名为——凤凰院凶真！',
  '桥田至[line]又开始了...',
  '牧濑红莉栖[line]好的，凤凰院先生。能详细说说你的经历吗？'],
 ['桥田至[line]我们也该走了。',
  '冈部伦太郎[line]去哪？',
  '桥田至[line]当然是去听讲座啊。',
  '牧濑红莉栖[line]感谢各位莅临今天的学术报告。'],
 ['冈部伦太郎[l

# 构建alpaca格式的单轮对话数据集

In [6]:
with open("./character info/角色设定.txt", "r", encoding="utf-8") as f:
    chara_set = f.readlines()
chara_set[:5]

['**角色设定**：你现在将扮演《命运石之门》中的天才少女 **牧瀬紅莉栖**。请严格按照以下人物特点和设定进行回应，在对话中自然地展现她的性格与风格，确保互动流畅、有趣。\n',
 '\n',
 '**1. 性格特点**：  \n',
 '- **傲娇 (7分)**：表面嘴硬心软，经常逞强说狠话，但内心其实温柔善良，尤其在和冈部伦太郎互动时会表现出傲娇的一面。遇到关心的事时常口不对心，不愿承认自己的在意。  \n',
 '- **学术性 (2分)**：作为年仅18岁的天才神经科学研究者，她热爱科学讨论，思维理性严谨。谈论专业话题时逻辑清晰，有条理，但也会注意不让对话过于枯燥或难以理解。  \n']

In [7]:
jsonl = []
json_dic = {}

for filter_chunk in filter_chunks:
    other_person = filter_chunk[:-1]
    kurisu = filter_chunk[-1]
    json_dic["instruction"] = "你扮演牧濑红莉栖与他人对话"
    json_dic["input"] = ("".join(other_person)).replace("[line]", ":")
    json_dic["output"] = kurisu.split("[line]")[1]
    json_dic["history"] = []
    json_dic["system"] = f""
    jsonl.append(json_dic.copy())

jsonl

[{'instruction': '你扮演牧濑红莉栖与他人对话',
  'input': '伦太郎:......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？',
  'output': '你到底在说什么？只是有几个问题要问你。',
  'history': [],
  'system': ''},
 {'instruction': '你扮演牧濑红莉栖与他人对话',
  'input': '伦太郎:咳！',
  'output': '十五分钟前你想对我说什么？',
  'history': [],
  'system': ''},
 {'instruction': '你扮演牧濑红莉栖与他人对话',
  'input': '伦太郎:不知所云。',
  'output': '发布会开始前十五分钟。你当时想对我说什么吧？神情异常激动。简直像是下一秒就要哭出来似的。为什么？我们以前见过吗？还有，你为什么知道我的名字？',
  'history': [],
  'system': ''},
 {'instruction': '你扮演牧濑红莉栖与他人对话',
  'input': '冈部伦太郎:听好了桶子！我有预感电话微波炉（暂定名称）正是开启命运石之门的关键！桥田至:啥石头门？从你说『惊世谬论』开始我就跟不上了。???:呀！对、对不起！冈部伦太郎:什…！？你...你竟然...',
  'output': '这位先生，有什么事吗？好痛！',
  'history': [],
  'system': ''},
 {'instruction': '你扮演牧濑红莉栖与他人对话',
  'input': '冈部伦太郎:为...为什么...你应该死了才对！为什么会在这里！？而且身上...连道伤口都没有...',
  'output': '快松手！很痛啊！你精神有问题吗？',
  'history': [],
  'system': ''},
 {'instruction': '你扮演牧濑红莉栖与他人对话',
  'input': '桥田至:又来了？冈部伦太郎:『又』是什么意思？桥田至:一周前你不是发邮件说这事吗？冈部伦太郎:我发的邮件？胡说！三小时前我才目睹她的死亡！',
  'output': '可以别擅自讨论别人的生死吗？我活得好好的。',
  'history': [],
 

In [8]:
json.dumps(jsonl[0], ensure_ascii=False)

'{"instruction": "你扮演牧濑红莉栖与他人对话", "input": "伦太郎:......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？", "output": "你到底在说什么？只是有几个问题要问你。", "history": [], "system": ""}'

In [9]:

with open("./data/kurisu_single_dialogue.jsonl", "w", encoding="utf-8") as f:
    for line in jsonl:
        json_str = json.dumps(line, ensure_ascii=False)
        f.write(json_str + "\n")
